In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Inizializzazione del dataset

Il dataset è composto da due sotto file in formato .csv:
1. trainingData, è un dataframe pandas creato a partire dal file di training "train.csv"
2. testData, è un dataframe pandas creato a partire dal file di testing "test.csv"


In [ ]:
trainData = pd.read_csv('train.csv')

In [ ]:
testData = pd.read_csv('test.csv')

## Controllo iniziale del TrainData set

Una volta caricato il TrainData set controllo che i dati presenti all'interno siano corretti, ovvero che rispettino le seguenti caratteristiche:

1. Assenza di campi nulli o NaN
2. Assenza di duplicati

In [ ]:
missing_data = trainData.isnull().sum(axis=0).reset_index()

In [ ]:
missing_data

In [ ]:
NaN_data = trainData.isna().sum(axis=0).reset_index()

In [ ]:
NaN_data

In [ ]:
duplicates = trainData.duplicated(['AppointmentID'], keep=False).sum(axis=0)

In [ ]:
print(duplicates)

## Controllo iniziale del TestData set

Una volta caricato il TestData set controllo che i dati presenti all'interno siano corretti, ovvero che rispettino le seguenti caratteristiche:

1. Assenza di campi nulli o NaN
2. Assenza di duplicati

In [ ]:
missing_data = testData.isnull().sum(axis=0).reset_index()

In [ ]:
missing_data

In [ ]:
NaN_data = testData.isna().sum(axis=0).reset_index()

In [ ]:
NaN_data

In [ ]:
duplicates = testData.duplicated(['AppointmentID'], keep=False).sum(axis=0)

In [ ]:
print(duplicates)

## Controllo dettagliato del trainData set

Una volta controllata la presenza di valori nulli o duplicati all'interno del dataset è possibile analizzarne nel dettaglio le varie colonne. Questo controllo supporta tutte le operazioni successive, dalla correzione dei dati errati fino alla feature engineering. 

Per semplificare l'analisi dei dati trasformo "No-show" da "Yes\No" a 1\0.

In [ ]:
trainData['No-show'].replace("No", 0, inplace=True)
trainData['No-show'].replace("Yes", 1, inplace=True)

In [ ]:
trainData.info()

In [ ]:
print(sorted(trainData.Age.unique()))

In [ ]:
sns.lineplot(x="Age", y="No-show", data=trainData)

In [ ]:
print(sorted(trainData.Neighbourhood.unique()))

In [ ]:
sns.lineplot(x="Neighbourhood", y="No-show", data=trainData)

In [ ]:
print(trainData.Handcap.unique())

In [ ]:
sns.lineplot(x="Handcap", y="No-show", data=trainData)

In [ ]:
print(trainData['No-show'].unique())

## Controllo dettagliato del testData set

Una volta controllata la presenza di valori nulli o duplicati all'interno del dataset è possibile analizzarne nel dettaglio le varie colonne. Questo controllo supporta tutte le operazioni successive, dalla correzione dei dati errati fino alla feature engineering. 

Per semplificare l'analisi dei dati trasformo "No-show" da "Yes\No" a 1\0.


In [ ]:
testData['No-show'].replace("No", 0, inplace=True)
testData['No-show'].replace("Yes", 1, inplace=True)

In [ ]:
testData.info()

## Correzione del trainData

Il trainData set potrebbe contenere dei dati non conformi alle analisi successive, come ad esempio dei campi vuoti, per questo è necessario compiere delle operazioni preliminari di correzione e normalizzazione. Le operazioni si concentrano sulle feature più importanti che saranno oggetto di valutazione successiva

### Correzione feature Age

Dall'analisi del dataset si nota come alcuni pazienti superano i 100 anni di età, potrebbe essere un errore di raccolta dei dati. Come prima operazione andiamo ad eliminare i valori di età inferiori a 0, che sicuramente sono errori, e quelli superiori a 100.

Successivamente verrà effettuato un controllo sulla colonna, se contiene valori nulli allora questi verranno rimpiazzati con la mediana della colonna riempendo i campi vuoti.

In [ ]:
trainData = trainData[(trainData.Age >= 0) & (trainData.Age <= 100)]

In [ ]:
if trainData.Age.isnull().sum(axis=0) > 0:
    print(trainData.Age.isnull().sum(axis=0))
    dataset = trainData.Age.fillna(trainData.Age.median())

### Correzione feature Gender

Anche il Gender potrebbe contenere dei valori nulli, per evitarlo riempiamo i campi vuoti con il valore 0. Probabilmente questa scelta introdurrà un piccolo bias all'interno del dataset, ma ci eviterà di effettuare analisi su una feature incompleta.

In [ ]:
if trainData.Gender.isnull().sum(axis=0) > 0:
    print(trainData.Gender.isnull().sum(axis=0))
    trainData = trainData.Gender.fillna(0)

### Correzione feature No-show

La colonna No-show è la colonna target della nostra classificazione, non possiamo avere dei campi nulli o non consistenti al suo interno. Aggiungendo dei valori 0 o 1 rischiamo di introdurre un bias troppo elevato, per questo eliminiamo eventuali righe contenenti valori nulli di No-show.

In [ ]:
if trainData["No-show"].isnull().sum(axis=0) > 0:
    print(trainData["No-show"].isnull().sum(axis=0))
    trainData = trainData["No-show"].dropna()

### Correzione features ScheduledDay, AppointmentDay e PatientID

Queste feature sono utilizzate durante la fase di feature engineering per la creazione di nuove feature, è dunque necessario che queste non siano vuote. Nel caso in cui si incontrassero delle righe contenenti valori inconsistenti si può solo che ricorrere alla loro eliminazione, poiché l'inserimento di valori a posteriori potrebbe creare dei bias o delle problematiche durante l'addestramento.

In [ ]:
if trainData.ScheduledDay.isnull().sum(axis=0) > 0:
    print(trainData.ScheduledDay.isnull().sum(axis=0))
    trainData = trainData.ScheduledDay.dropna()

In [ ]:
if trainData.AppointmentDay.isnull().sum(axis=0) > 0:
    print(trainData.AppointmentDay.isnull().sum(axis=0))
    trainData = trainData.AppointmentDay.dropna()

In [ ]:
if trainData.PatientId.isnull().sum(axis=0) > 0:
    print(trainData.PatientId.isnull().sum(axis=0))
    trainData = trainData.PatientId.dropna()

## Correzione del testData

Il testData set potrebbe contenere dei dati non conformi alle analisi successive, come ad esempio dei campi vuoti, per questo è necessario compiere delle operazioni preliminari di correzione e normalizzazione. 

Le operazioni si concentrano sulle feature più importanti che saranno oggetto di valutazione successiva. Le motivazioni sull'utilizzo di queste funzioni di correzioni sono le stesse di quelle sul training set. Ometto i commenti relativi per ridurre la dimensione del notebook.

### Correzione feature Age


In [ ]:
testData = testData[(testData.Age >= 0) & (testData.Age <= 100)]

In [ ]:
if testData.Age.isnull().sum(axis=0) > 0:
    print(testData.Age.isnull().sum(axis=0))
    dataset = testData.Age.fillna(testData.Age.median())

### Correzione feature Gender


In [ ]:
if testData.Gender.isnull().sum(axis=0) > 0:
    print(testData.Gender.isnull().sum(axis=0))
    testData = testData.Gender.fillna(0)

### Correzione feature No-show


In [ ]:
if testData["No-show"].isnull().sum(axis=0) > 0:
    print(testData["No-show"].isnull().sum(axis=0))
    testData = testData["No-show"].dropna()

### Correzione features ScheduledDay, AppointmentDay e PatientID


In [ ]:
if testData.ScheduledDay.isnull().sum(axis=0) > 0:
    print(testData.ScheduledDay.isnull().sum(axis=0))
    testData = testData.ScheduledDay.dropna()

In [ ]:
if testData.AppointmentDay.isnull().sum(axis=0) > 0:
    print(testData.AppointmentDay.isnull().sum(axis=0))
    testData = testData.AppointmentDay.dropna()

In [ ]:
if testData.PatientId.isnull().sum(axis=0) > 0:
    print(testData.PatientId.isnull().sum(axis=0))
    testData = testData.PatientId.dropna()

## Feature Engineering

Osservando le informazioni sul dataset è possibile andare a compiere delle modifiche sulle varie features per migliorarne l'usabilità e la comprensione. Inoltre verranno eliminate PatientID e AppointmentID poiché poco rilevanti.

Per semplificare l'analisi dei dati trasformo "Gender" da "F\M" a 0\1.

In [ ]:
trainData['Gender'].replace("F", 0, inplace=True)
trainData['Gender'].replace("M", 1, inplace=True)

In [ ]:
testData['Gender'].replace("F", 0, inplace=True)
testData['Gender'].replace("M", 1, inplace=True)

### Feature WaitingDays

Osservando il dataset posso assumere che ScheduledDay e AppointmentDay sono due feature importanti, probabilmente molto correlate con la variabile target No-show. 

Utilizzo gli strumenti offerti da NumPy e Pandas al fine di creare una nuova colonna, che chiamerò WaitingDays, che conterrà il numero di giorni che un paziente attende prima di poter essere visitato. 

Questa scelta è motivata dal fatto che potrebbe esserci una correlazione tra i giorni di attesa e il non presentarsi ad un appuntamento, magari l'assenza è frutto di una dimenticanza data dal tempo. 

Dalla analisi del grafo sembrerebbe che il numero di giorni di attesa incidano molto sul presentarsi o meno ad un appuntamento, probabilmente a causa di una dimenticanza dovuta dalla troppa attesa, soprattutto al superamento dei 100 giorni. 

### TrainingSet

In [ ]:
trainData[['ScheduledDay', 'AppointmentDay']] = trainData[['ScheduledDay', 'AppointmentDay']].apply(pd.to_datetime)

In [ ]:
trainData['WaitingDays'] = trainData["AppointmentDay"].sub(trainData["ScheduledDay"], axis=0)

In [ ]:
trainData["WaitingDays"] = (trainData["WaitingDays"]).abs().dt.days

In [ ]:
trainData = trainData.drop(columns=['ScheduledDay', 'AppointmentDay'])

In [ ]:
sns.lineplot(x="WaitingDays", y="No-show", data=trainData)

### TestData

In [ ]:
testData[['ScheduledDay', 'AppointmentDay']] = testData[['ScheduledDay', 'AppointmentDay']].apply(pd.to_datetime)

In [ ]:
testData['WaitingDays'] = testData["AppointmentDay"].sub(testData["ScheduledDay"], axis=0)

In [ ]:
testData["WaitingDays"] = (testData["WaitingDays"]).abs().dt.days

In [ ]:
testData = testData.drop(columns=['ScheduledDay', 'AppointmentDay'])

### Feature MissedPastAppointment 

Oltre che i giorni di attesa prima di essere visitati è utile andare ad analizzare il numero di appuntamenti nei quali un paziente non si è presentato. Questa analisi è motivata dal fatto che potrebbe esserci una correlazione tra il numero di appuntamenti saltati nel passato e il non presentarsi in futuro. 

La feature viene realizzatra raggruppando per PatientId e sommando il numero di No-show associati. Questo è supportato dal fatto che la feature PatientId contiene dei duplicati, ovvero ci sono più prenotazioni effettuate dallo stesso paziente per appuntamenti diversi

Dall'analisi del grafo risulta che vi è un alta correlazione tra il numero di appuntamenti saltati nel passato e il non presentarsi ad un successivo appuntamento.

In [ ]:
trainData['MissedPastAppointments'] = trainData.groupby('PatientId')['No-show'].apply(lambda x: x.cumsum())

In [ ]:
sns.lineplot(x="MissedPastAppointments", y="No-show", data=trainData)

In [ ]:
trainData = trainData.drop(columns=['PatientId', 'AppointmentID'])

In [ ]:
testData['MissedPastAppointments'] = testData.groupby('PatientId')['No-show'].apply(lambda x: x.cumsum())

In [ ]:
testData = testData.drop(columns=['PatientId', 'AppointmentID'])

### LabelEncoding 

Il dataset contiene valori non numerici, è necessario trasformare tutte le feature in valori numerici per renderli compatibili con gli algoritmi di classificazione

La funzione Label Encoding permette di codificare tutti i dati non numerici, i label appunto, in valori compresi tra 0 e N-1 dove N sono il numero di valori univoci presenti all'interno delle colonne

Le colonne da sistemare vengono ricavate dalla struttura dati X, che contiene l'intero dataset

La funzione lambda permette di applicare la funzione di traformazione offerta da LabelEncoder su tutte le colonne da sistemare. 

Dopo l'esecuzione dell'encoding viene stampata la tabella contenente le colonne trasformate, che potrebbe omettere alcune colonne presenti all'interno del dataset poiché non soggette ad encoding, e la tabella completa per visualizzare il contenuto di X


In [ ]:
from sklearn.preprocessing import LabelEncoder

### TrainingSet

In [ ]:
feature_mask = trainData.dtypes==object
columns = trainData.columns[feature_mask].tolist()

In [ ]:
le = LabelEncoder()
trainData[columns] = trainData[columns].apply(lambda col: le.fit_transform(col))

In [ ]:
trainData

### TestSet

In [ ]:
feature_mask = testData.dtypes==object
columns = testData.columns[feature_mask].tolist()

In [ ]:
le = LabelEncoder()
testData[columns] = testData[columns].apply(lambda col: le.fit_transform(col))

In [ ]:
testData

## Inizializzazione Modello 

Il modello scelto è un classificatore RandomForest. Il classificatore RandomForest utilizza diversi classificatori Decision Tree su diversi sotto campioni del dataset di input. Ogni classificatore compierà un'operazione di fitting sul suo sottocampione. Il classificatore RandomForest utilizzerà una funzione di averaging, ovvero fa la media tra le varie predizioni, per migliorare l'accuratezza e tenere sotto controllo l'overfitting. 

Un Decision tree classifier ha il compito di predirre il valore della variabile target, nel nostro caso Churn, apprendendo delle regole di decisione a partire dalle feature del nostro dataset

In [ ]:
datasetTrain = trainData.drop(columns = ['No-show'])

In [ ]:
datasetTest = testData.drop(columns = ['No-show'])

In [ ]:
y_train = trainData["No-show"]

In [ ]:
y_test = testData["No-show"]

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
clf.fit(datasetTrain, y_train)

In [ ]:
predict_TestSet = clf.predict(datasetTest)

In [ ]:
predict_TrainingSet = clf.predict(datasetTrain)

In [ ]:
print("Training set accuracy: {:.2f}".format(accuracy_score(y_train, predict_TrainingSet)))

In [ ]:
print("Test set accuracy: {:.2f}".format(accuracy_score(y_test, predict_TestSet)))